In [ ]:
import nglview
import ipywidgets

pdb_code = "2VGB"
ligand_code = "PGA"
api_url = "http://mmb.irbbarcelona.org/api/pdb/"
mgltools_path = "/anaconda3/pkgs/mgltools-1.5.7-0"

In [ ]:
from biobb_io.api.pdb import Pdb

In [ ]:
download_pdb = "download.pdb"
prop = {
  "pdb_code": pdb_code,
  "filter": ["ATOM", "HETATM"],
  "url": api_url
}

Pdb(output_pdb_path=download_pdb,
    properties=prop).launch()

In [ ]:
view = nglview.show_structure_file(download_pdb, default=False)
view.add_representation(repr_type='cartoon', 
                        selection='not het',
                       colorScheme = 'atomindex')
view.add_representation(repr_type='ball+stick', 
                        selection=ligand_code,
                       aspectRatio = 5)
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

In [ ]:
from biobb_structure_utils.utils.extract_chain import ExtractChain

In [ ]:
extract_chain = "extract_chain.pdb"
prop = {
    "chains": ["A"]
}

ExtractChain(input_structure_path=download_pdb,
             output_structure_path = extract_chain,
            properties=prop).launch()

In [ ]:
view = nglview.show_structure_file(extract_chain, default=False)
view.add_representation(repr_type='cartoon', 
                        selection='not het',
                        colorScheme = 'atomindex')
view.add_representation(repr_type='ball+stick', 
                        selection=ligand_code,
                       aspectRatio = 5)
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

In [ ]:
from biobb_io.api.pdb_cluster_zip import MmbPdbClusterZip

In [ ]:
pdb_cluster = "pdb_cluster.zip"
prop = {
    "pdb_code": pdb_code,
    "filter": ["ATOM", "HETATM"],
    "cluster": 90,
    "url": api_url
}

MmbPdbClusterZip(output_pdb_zip_path = pdb_cluster,
            properties=prop).launch()

In [ ]:
from biobb_vs.utils.bindingsite import BindingSite

In [ ]:
bindingsite = "bindingsite.pdb"
prop = {
    "ligand": ligand_code,
    "matrix_name": "blosum62",
    "gap_open": -10.0,
    "gap_extend": -0.5,
    "max_num_ligands": 15,
    "radius": 5
}

BindingSite(input_pdb_path = extract_chain,
            input_clusters_zip = pdb_cluster,
            output_pdb_path = bindingsite,
            properties=prop).launch()

In [ ]:
view = nglview.show_structure_file(extract_chain, default=False)
view[0].add_representation(repr_type='cartoon', 
                        selection='not het',
                          opacity=.2,
                          color='#cccccc')

view.add_component(bindingsite, default=False)
view[1].add_representation(repr_type='surface', 
                           selection='not ' + ligand_code, 
                           opacity = .3)
# ligand
view[1].add_representation(repr_type='licorice', 
                        selection='not ' + ligand_code)
# ligand
view[1].add_representation(repr_type='ball+stick', 
                        selection=ligand_code,
                       aspectRatio = 4)

view[0].center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

In [ ]:
from biobb_vs.utils.box import Box

In [ ]:
box = "box.pdb"
prop = {
    "offset": 2,
    "box_coordinates": True
}

Box(input_pdb_path = extract_chain,
            resid_pdb_path = bindingsite,
            output_pdb_path = box,
            properties=prop).launch()

In [ ]:
view = nglview.show_structure_file(box, default=False)

atomPair = [
    [ "9999:Z.ZN1", "9999:Z.ZN2" ],
    [ "9999:Z.ZN2", "9999:Z.ZN4" ],
    [ "9999:Z.ZN4", "9999:Z.ZN3" ],
    [ "9999:Z.ZN3", "9999:Z.ZN1" ],
    
    [ "9999:Z.ZN5", "9999:Z.ZN6" ],
    [ "9999:Z.ZN6", "9999:Z.ZN8" ],
    [ "9999:Z.ZN8", "9999:Z.ZN7" ],
    [ "9999:Z.ZN7", "9999:Z.ZN5" ],
    
    [ "9999:Z.ZN1", "9999:Z.ZN5" ],
    [ "9999:Z.ZN2", "9999:Z.ZN6" ],
    [ "9999:Z.ZN3", "9999:Z.ZN7" ],
    [ "9999:Z.ZN4", "9999:Z.ZN8" ]
]

#view.shape.add_cylinder( [ 0, 2, 7 ], [ 10, 0, 9 ], [ 1, 0, 0 ], 0.1 )

# structure
view.add_representation(repr_type='cartoon', 
                        selection='not het',
                        color='#cccccc',
                       opacity=.2)
# ligands box
view.add_representation(repr_type='ball+stick', 
                        selection='9999',
                       aspectRatio = 10)
# lines box
view.add_representation(repr_type='distance', 
                        atomPair= atomPair,
                       labelColor= 'transparent',
                       color= 'black')

view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

In [ ]:
from biobb_structure_utils.utils.extract_heteroatoms import ExtractHeteroAtoms

In [ ]:
ligand = "ligand.pdb"
prop = {
    "heteroatoms": [{ "name": ligand_code }]
}

ExtractHeteroAtoms(input_structure_path=extract_chain,
output_heteroatom_path=ligand,
properties=prop).launch()

In [ ]:
from biobb_chemistry.ambertools.reduce_add_hydrogens import ReduceAddHydrogens

In [ ]:
ligand_H = "ligand_H.pdb"
prop = {
    
}

ReduceAddHydrogens(input_path=ligand,
output_path=ligand_H,
properties=prop).launch()

In [ ]:
from ipywidgets import HBox

In [ ]:
v0 = nglview.show_structure_file(ligand)
v1 = nglview.show_structure_file(ligand_H)

v0._set_size('500px', '')
v1._set_size('500px', '')

def on_change(change):
    v1._set_camera_orientation(change['new'])
    
v0.observe(on_change, ['_camera_orientation'])

HBox([v0, v1])

In [ ]:
#from biobb_vs.vina.vina_prepare_ligand import VinaPrepareLigand

In [ ]:
#prep_ligand = "prep_ligand.pdbqt"
#prop = {
#    "mgltools_path": mgltools_path
#}

#VinaPrepareLigand(input_ligand_path = ligand_H,
#            output_ligand_path = prep_ligand,
#            properties=prop).launch()

In [ ]:
from biobb_chemistry.babelm.babel_convert import BabelConvert

In [ ]:
prep_ligand = "prep_ligand.pdbqt"
prop = {
    "input_format": "pdb",
    "output_format": "pdbqt",
    "obabel_path": "/anaconda3/bin/obabel"
}

BabelConvert(input_path = ligand_H,
            output_path = prep_ligand,
            properties=prop).launch()

In [ ]:
from biobb_structure_utils.utils.remove_ligand import RemoveLigand

In [ ]:
receptor = "receptor.pdb"
prop = {
    "ligand": ligand_code
}

RemoveLigand(input_structure_path=extract_chain,
             output_structure_path = receptor,
            properties=prop).launch()

In [ ]:
#from biobb_vs.vina.vina_prepare_receptor import VinaPrepareReceptor

In [ ]:
#prep_receptor = "prep_receptor.pdbqt"
#prop = {
#    "mgltools_path": mgltools_path
#}

#VinaPrepareReceptor(input_receptor_path = receptor,
#            output_receptor_path = prep_receptor,
#            properties=prop).launch()

In [ ]:
#from biobb_structure_utils.utils.remove_pdb_water import RemovePdbWater

In [ ]:
#receptor_no_wat = "receptor_no_wat.pdb"

#RemovePdbWater(input_pdb_path = receptor,
#             output_pdb_path = receptor_no_wat).launch()

In [ ]:
from biobb_structure_utils.utils.str_check_add_hydrogens import StrCheckAddHydrogens

In [ ]:
prep_receptor = "prep_receptor.pdbqt"
prop = {
    "charges": True,
    "mode": "auto"
}

StrCheckAddHydrogens(input_structure_path = receptor,
            output_structure_path = prep_receptor,
            properties=prop).launch()

In [ ]:
# if prep_receptor ends with END, remove last line
lines_new = []
with open(prep_receptor, 'r') as f:
    lines = f.read().splitlines()
    for item in lines:
        print(item)
        if not item.startswith('END'):
            lines_new.append(item)
        
with open(prep_receptor, 'w') as f:
    for item in lines_new:
        f.write("%s\n" % item)


In [ ]:
from biobb_vs.vina.autodock_vina import AutoDockVina

In [ ]:
output_vina_pdbqt = "output_vina.pdbqt"
output_vina_log = "output_vina.log"

AutoDockVina(input_ligand_pdbqt_path = prep_ligand,
             input_receptor_pdbqt_path = prep_receptor,
             input_box_path = box,
            output_pdbqt_path = output_vina_pdbqt,
             output_log_path = output_vina_log).launch()

In [ ]:
models = 'all'
#models = '/0 or /1 or /4'

v0 = nglview.show_structure_file(output_vina_pdbqt, default=False)
v0.add_representation(repr_type='licorice', 
                        selection=models,
                       colorScheme= 'partialCharge')
v0.center()
v1 = nglview.show_structure_file(output_vina_pdbqt, default=False)
v1.add_representation(repr_type='ball+stick', 
                        selection=models)
v1.center()

v0._set_size('500px', '')
v1._set_size('500px', '')

def on_change(change):
    v1._set_camera_orientation(change['new'])
    
v0.observe(on_change, ['_camera_orientation'])

HBox([v0, v1])

In [ ]:
from biobb_vs.utils.extract_model_pdbqt import ExtractModelPDBQT

In [ ]:
output_pdbqt_model = "output_model.pdbqt"
prop = {
    "model": 1
}

ExtractModelPDBQT(input_pdbqt_path = output_vina_pdbqt,
             output_pdbqt_path = output_pdbqt_model,
            properties=prop).launch()

In [ ]:
v0 = nglview.show_structure_file(ligand)
v1 = nglview.show_structure_file(output_pdbqt_model)

v0._set_size('500px', '')
v1._set_size('500px', '')

def on_change(change):
    v1._set_camera_orientation(change['new'])
    
v0.observe(on_change, ['_camera_orientation'])

HBox([v0, v1])

In [ ]:
#v0 = nglview.show_structure_file(extract_chain)

#v1 = nglview.show_structure_file(receptor, default=False)
#v1[0].add_representation(repr_type='cartoon', 
#                        selection='not het')
#v1.add_component(output_pdbqt_model, default=False)
#v1[1].add_representation(repr_type='ball+stick', 
#                        selection='all',
#                       aspectRatio = 5)
#v1[0].center()

#v0._set_size('500px', '500px')
#v1._set_size('500px', '500px')

#def on_change(change):
#    v1._set_camera_orientation(change['new'])
    
#v0.observe(on_change, ['_camera_orientation'])

#HBox([v0, v1])

In [ ]:
from biobb_chemistry.babelm.babel_convert import BabelConvert

In [ ]:
output_pdb_model = "output_model.pdb"
prop = {
    "input_format": "pdbqt",
    "output_format": "pdb",
    "obabel_path": "/anaconda3/bin/obabel"
}

BabelConvert(input_path = output_pdbqt_model,
             output_path = output_pdb_model,
            properties=prop).launch()

In [ ]:
from biobb_structure_utils.utils.cat_pdb import CatPDB

In [ ]:
output_structure = "output_structure.pdb"

CatPDB(input_structure1 = receptor,
             input_structure2 = output_pdb_model,
             output_structure_path = output_structure).launch()

In [ ]:
v0 = nglview.show_structure_file(extract_chain)

v1 = nglview.show_structure_file(output_structure)

v0._set_size('500px', '500px')
v1._set_size('500px', '500px')

def on_change(change):
    v1._set_camera_orientation(change['new'])
    
v0.observe(on_change, ['_camera_orientation'])

HBox([v0, v1])